### HomeWork 1

Unigrams, bigrams, and in general n-grams are 1,2 or n words that appear consecutively in a single sentence. Consider the sentence:

    "to know you is to love you."

This sentence contains:

    Unigrams(single words): to(2 times), know(1 time), you(2 times), is(1 time), love(1 time)
    Bigrams: "to know","know you","you is", "is to","to love", "love you" (all 1 time)
    Trigrams: "to know you", "know you is", "you is to", "is to love", "to love you" (all 1 time)

The goal of this HW is to find the most common n-grams in the text of Moby Dick.

Your task is to:

* Convert all text to lower case, remove all punctuations. (Finally, the text should contain only letters, numbers and spaces)
* Count the occurance of each word and of each 2,3,4,5 - gram
* List the 5 most common elements for each order (word, bigram, trigram...). For each element, list the sequence of words and the number of occurances.

Basically, you need to change all punctuations to a space and define as a word anything that is between whitespace or at the beginning or the end of a sentence, and does not consist of whitespace (strings consisiting of only white spaces should not be considered as words). The important thing here is to be simple, not to be 100% correct in terms of parsing English. Evaluation will be primarily based on identifying the 5 most frequent n-grams in correct order for all values of n. Some slack will be allowed in the values of frequency of ngrams to allow flexibility in text processing.   

This text is short enough to process on a single core using standard python. However, you are required to solve it using RDD's for the whole process. At the very end you can use `.take(5)` to bring the results to the central node for printing.

The code for reading the file and splitting it into sentences is shown below:

In [1]:
!cd ~/Documents/youtube/DSE230_Data_Analysis_Using_Hadoop_and_Spark_UCSD

textRDD = sc.newAPIHadoopFile('/data/Moby-Dick.txt',
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                               conf={'textinputformat.record.delimiter': "\r\n\r\n"}) \
            .map(lambda x: x[1])

sentences=textRDD.flatMap(lambda x: x.split(". "))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.newAPIHadoopFile.
: org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input path does not exist: file:/data/Moby-Dick.txt
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:321)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:264)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.getSplits(FileInputFormat.java:385)
	at org.apache.spark.rdd.NewHadoopRDD.getPartitions(NewHadoopRDD.scala:113)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Option.scala:120)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org

Note: For running the file on cluster, change the file path to `'/data/Moby-Dick.txt'`

Let `freq_ngramRDD` be the final result RDD containing the n-grams sorted by their frequency in descending order. Use the following function to print your final output:

In [12]:
def printOutput(n,freq_ngramRDD):
    top=freq_ngramRDD.take(5)
    print '\n============ %d most frequent %d-grams'%(5,n)
    print '\nindex\tcount\tngram'
    for i in range(5):
        print '%d.\t%d: \t"%s"'%(i+1,top[i][0],' '.join(top[i][1]))

Your output for unigrams should look like:
```
============ 5 most frequent 1-grams

index	count	ngram
1.       40: 	 "a"
2.	   25: 	 "the"
3.	   21: 	 "and"
4.	   16: 	 "to"
5.	   9:  	 "of"

```
Note: This is just a sample output and does not resemble the actual results in any manner.

Your final program should generate an output using the following code:

In [23]:
import string
import re
regex = re.compile('[%s]' % re.escape(string.punctuation))

def test_re(s):  
    return regex.sub('', s)

# n grams using generator comprehension
def BetterNgrams(input_list, n):
    return zip(*(input_list[i:] for i in range(n)))

In [25]:
for n in xrange(1,6):
    # Put your logic for generating the sorted n-gram RDD here and store it in freq_ngramRDD variable
    
    freq_ngramRDD = sentences.map(lambda x: test_re(x)) \
                    .flatMap(lambda x: BetterNgrams(x.split(), n)) \
                    .map(lambda x: (x, 1)) \
                    .reduceByKey(lambda x,y:x+y) \
                    .map(lambda x:(x[1],x[0])) \
                    .sortByKey(False)
    printOutput(n,freq_ngramRDD)   


============ 5 most frequent 1-grams

index	count	ngram
1.	13787: 	"the"
2.	6619: 	"of"
3.	6029: 	"and"
4.	4571: 	"to"
5.	4539: 	"a"

============ 5 most frequent 2-grams

index	count	ngram
1.	1874: 	"of the"
2.	1134: 	"in the"
3.	728: 	"to the"
4.	432: 	"from the"
5.	370: 	"of his"

============ 5 most frequent 3-grams

index	count	ngram
1.	73: 	"of the whale"
2.	61: 	"one of the"
3.	60: 	"the Sperm Whale"
4.	57: 	"of the sea"
5.	53: 	"out of the"

============ 5 most frequent 4-grams

index	count	ngram
1.	25: 	"of the Sperm Whale"
2.	20: 	"at the same time"
3.	17: 	"the bottom of the"
4.	13: 	"Project Gutenberg Literary Archive"
5.	13: 	"Gutenberg Literary Archive Foundation"

============ 5 most frequent 5-grams

index	count	ngram
1.	13: 	"Project Gutenberg Literary Archive Foundation"
2.	11: 	"the Project Gutenberg Literary Archive"
3.	10: 	"and at the same time"
4.	9: 	"the bottom of the sea"
5.	7: 	"the terms of this agreement"
